In [1]:
import copy
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx
from ItalySetup import ItalySetup
from covidOCP import COVIDVaccinationOCP, COVIDParametersOCP
from main import pick_scenario, build_scenario
import seaborn as sns
import datetime
import pickle
import os

nx = 9
states_names = ['S', 'E', 'P', 'I', 'A', 'Q', 'H', 'R', 'V']
when = 'future'
file_prefix = f'week'
outdir = 'helvetios-runs/2021-02-02-107_30/'

generated_dir = 'model_output/scenarios_102_30'

n_int_steps = 6
nnodes = 107  # nodes
ndays_ocp = 30
ndays = 30

setup = ItalySetup(nnodes, ndays, when)
setup_ocp = ItalySetup(nnodes, ndays_ocp, when)
M = setup.nnodes
N = len(setup.model_days) - 1

with open(f'italy-data/parameters_{nnodes}_{when}.pkl', 'rb') as inp:
    p = pickle.load(inp)
    
os.makedirs(f'{generated_dir}', exist_ok=True)

scenarios = {pick_scenario(setup, i)['name']:pick_scenario(setup, i) for i in np.arange(16)}
scenarios.keys()

Loaded Italy Setup with 107 nodes.
Loaded Italy Setup with 107 nodes.


dict_keys(['U-r15-t125000-id0', 'L-r15-t125000-id1', 'U-r150-t125000-id2', 'L-r150-t125000-id3', 'U-r15-t250000-id4', 'L-r15-t250000-id5', 'U-r150-t250000-id6', 'L-r150-t250000-id7', 'U-r15-t479700-id8', 'L-r15-t479700-id9', 'U-r150-t479700-id10', 'L-r150-t479700-id11', 'U-r15-t1000000-id12', 'L-r15-t1000000-id13', 'U-r150-t1000000-id14', 'L-r150-t1000000-id15'])

In [2]:
# Choose a subset of scenarios:
#pick = 'L-r15'
#scenarios = {k:v for (k,v) in scenarios.items() if pick in k}
#print(len(scenarios))

In [4]:
scenarios_opt = {}
scenarios_baseline = {}
## Re-integrate vacc
for scenario_name, scenario in scenarios.items():
    fname = f"{outdir}{file_prefix}-{scenario_name}-opt-{nnodes}_{ndays_ocp}.csv"  # '-'.join(scenario_name.split('-')[:-1])
    try:
        md = pd.read_csv(fname, index_col= 'date', parse_dates=True)
        print(f'YES {fname}')
    
        # Build scenario
        maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
        M = setup.nnodes
        N = setup.ndays - 1
        control_initial = np.zeros((M, N))
        unvac_nd = np.copy(setup.pop_node)
        stockpile = 0
        for k in range(ndays_ocp - 1):
            stockpile += delivery_national[k]
            for nodename in md.place.unique():
                nd = setup.ind2name.index(nodename)
                to_allocate = md[(md['place'] == nodename) & (md['comp'] == 'vacc')].iloc[k]['value']
                to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
                control_initial[nd, k] = to_allocate
                stockpile -= to_allocate
                unvac_nd[nd] -= to_allocate
        p.apply_epicourse(setup, scenario['beta_mult'])
        # END Build scenario

        results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                          setup=setup,
                                                                          parameters=p,
                                                                          controls=control_initial,
                                                                          save_to=f'{generated_dir}/{scenario_name}-opi-{nnodes}_{ndays}',
                                                                          n_rk4_steps=n_int_steps)
        results.set_index('date', drop=True, inplace=True)
        scenarios_opt[scenario_name] = scenario
        print(f'--> DONE {scenario_name}')

        if scenario_name.split('-')[0] not in scenarios_baseline:
            control_initial = np.zeros((M, N))
            # Generate NO vaccination scenarios
            results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-novacc-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
            scenarios_baseline[scenario_name.split('-')[0]] = pd.read_csv(f'{generated_dir}/{scenario_name}-novacc-{nnodes}_{ndays}.csv', 
                                                                         index_col= 'date', parse_dates=True)


    except FileNotFoundError:
        print(f'NOT {fname}')

YES helvetios-runs/2021-02-02-107_30/week-U-r15-t125000-id0-opt-107_30.csv


 10%|█         | 3/29 [00:00<00:01, 20.62it/s]

===> Integrating for model_output/scenarios_102_30/U-r15-t125000-id0-opi-107_30


  7%|▋         | 2/29 [00:00<00:01, 18.59it/s]

--> DONE U-r15-t125000-id0
===> Integrating for model_output/scenarios_102_30/U-r15-t125000-id0-novacc-107_30


...

--> DONE L-r15-t1000000-id13
YES helvetios-runs/2021-02-02-107_30/week-U-r150-t1000000-id14-opt-107_30.csv


  7%|▋         | 2/29 [00:00<00:01, 18.86it/s]

===> Integrating for model_output/scenarios_102_30/U-r150-t1000000-id14-opi-107_30


100%|██████████| 29/29 [00:01<00:00, 21.60it/s]


--> DONE U-r150-t1000000-id14
NOT helvetios-runs/2021-02-02-107_30/week-L-r150-t1000000-id15-opt-107_30.csv


In [5]:
# Generate alternatives:
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())

def generate_all_alternatives(scenario_name, scenario):
    # BY INCIDENCE: 
    nv = scenarios_baseline[scenario_name.split('-')[0]]
    incid = nv[nv['comp'].isin(['yell'])].groupby('placeID').sum()
    incid.sort_values('value', ascending=False)
    
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp-1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * incid.loc[nd]['value'] / incid['value'].sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-inc-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)

    # BY Suceptibility: 
    nv = scenarios_baseline[scenario_name.split('-')[0]]
    incid = nv[nv['comp'].isin(['S'])].loc[str(setup.start_date)]
    incid.set_index('placeID', inplace=True)
    
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp -1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * incid.loc[nd]['value'] / incid['value'].sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-sus-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)
    
    
    # BY POPULATION 
    # Build scenario
    maxvaccrate_regional, delivery_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
    M = setup.nnodes
    N = setup.ndays - 1
    control_initial = np.zeros((M, N))
    unvac_nd = np.copy(setup.pop_node)
    stockpile = 0
    for k in range(ndays_ocp -1):
        stockpile += delivery_national[k]
        for nodename in md.place.unique():
            pop_nd = setup.pop_node[nd]
            nd = setup.ind2name.index(nodename)
            to_allocate = stockpile * pop_nd / setup.pop_node.sum()
            to_allocate = min(to_allocate, maxvaccrate_regional[nd, k], unvac_nd[nd], stockpile)
            control_initial[nd, k] = to_allocate
            stockpile -= to_allocate
            unvac_nd[nd] -= to_allocate
    p.apply_epicourse(setup, scenario['beta_mult'])
    # END Build scenario

    results, state_initial, yell, mob = COVIDVaccinationOCP.integrate(N,
                                                                      setup=setup,
                                                                      parameters=p,
                                                                      controls=control_initial,
                                                                      save_to=f'{generated_dir}/{scenario_name}-pop-{nnodes}_{ndays}',
                                                                      n_rk4_steps=n_int_steps)
    results.set_index('date', drop=True, inplace=True)

    #TODO: Centrality based and R0 based
    return True
    
all_sims = pool.starmap(generate_all_alternatives, [(scenario_name, scenario) for scenario_name, scenario in scenarios_opt.items()])

===> Integrating for model_output/scenarios_102_30/L-r150-t479700-id11-inc-107_30===> Integrating for model_output/scenarios_102_30/U-r150-t250000-id6-inc-107_30===> Integrating for model_output/scenarios_102_30/U-r15-t250000-id4-inc-107_30


===> Integrating for model_output/scenarios_102_30/U-r150-t1000000-id14-inc-107_30===> Integrating for model_output/scenarios_102_30/L-r150-t250000-id7-inc-107_30


  0%|          | 0/29 [00:00<?, ?it/s]

===> Integrating for model_output/scenarios_102_30/U-r15-t479700-id8-inc-107_30


  0%|          | 0/29 [00:00<?, ?it/s]


===> Integrating for model_output/scenarios_102_30/U-r150-t479700-id10-inc-107_30

...

  3%|▎         | 1/29 [00:00<00:03,  8.86it/s]

===> Integrating for model_output/scenarios_102_30/U-r150-t1000000-id14-pop-107_30


  7%|▋         | 2/29 [00:00<00:03,  8.57it/s]

===> Integrating for model_output/scenarios_102_30/U-r150-t250000-id6-pop-107_30


100%|██████████| 29/29 [00:03<00:00,  8.22it/s]


# Generate the averted cases for all posterior draws

In [6]:
import matlab.engine

eng = matlab.engine.start_matlab()
eng.cd('check-strats/', nargout=0)

eng.workspace['timesV'] = matlab.double(np.arange(738160, 738160+ndays).tolist()) # THIS IS FOR JANUARY 4st

In [7]:
scenarios = {pick_scenario(setup, i)['name']:pick_scenario(setup, i) for i in np.arange(16)}
scenarios.keys()

dict_keys(['U-r15-t125000-id0', 'L-r15-t125000-id1', 'U-r150-t125000-id2', 'L-r150-t125000-id3', 'U-r15-t250000-id4', 'L-r15-t250000-id5', 'U-r150-t250000-id6', 'L-r150-t250000-id7', 'U-r15-t479700-id8', 'L-r15-t479700-id9', 'U-r150-t479700-id10', 'L-r150-t479700-id11', 'U-r15-t1000000-id12', 'L-r15-t1000000-id13', 'U-r150-t1000000-id14', 'L-r150-t1000000-id15'])

In [8]:
scn_results = pd.DataFrame(columns=['newdoseperweek', 'method', 'infected'])
methods = ['opi','inc', 'pop', 'sus'] # 'opt'

for met in methods:
    for scenario_name, scenario in scenarios.items():
        filename = f'{generated_dir}/{scenario_name}-{met}-{nnodes}_{ndays}.csv'
        if os.path.isfile(filename):
            print(f'Doing {filename}')
            maxvaccrate_regional, stockpile_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
            p.apply_epicourse(setup, scenario['beta_mult'])
            
            md = pd.read_csv(filename, index_col= 'date', parse_dates=True)
            
            dosesV = md[md['comp'] == 'vacc'].pivot(columns='placeID', values='value').to_numpy()

            
            eng.workspace['Vdoses'] = matlab.double(dosesV.tolist())
            eng.workspace['beta_ratio'] = matlab.double(scenario['beta_mult'][0].tolist())
            eng.run('main_script.m', nargout=0)
            ens_exposed = np.array(eng.eval('ens_exposed_preprocess')).flatten()
        
            scn_results = pd.concat([scn_results, pd.DataFrame.from_dict({'newdoseperweek':[int(scenario_name.split('-')[2][1:])]*len(ens_exposed),
                                                                           'method': [met]*len(ens_exposed),
                                                                           'infected':ens_exposed.tolist(),
                                                                           'post_sample':np.arange(len(ens_exposed)),
                                                                           'doses': [dosesV.sum()]*len(ens_exposed),
                                                                           'scenario-beta': [scenario_name.split('-')[0]]*len(ens_exposed),
                                                                           'scenario-rate': [scenario_name.split('-')[1]]*len(ens_exposed),
                                                                           'scenario-tot': [scenario_name.split('-')[2]]*len(ens_exposed),
                                                                           'scenario': [scenario_name]*len(ens_exposed)
                                                                         })])
        else:
            print(f'not found {scenario_name}')

Doing model_output/scenarios_102_30/U-r15-t125000-id0-opi-107_30.csv
Doing model_output/scenarios_102_30/L-r15-t125000-id1-opi-107_30.csv
Doing model_output/scenarios_102_30/U-r150-t125000-id2-opi-107_30.csv
Doing model_output/scenarios_102_30/L-r150-t125000-id3-opi-107_30.csv
Doing model_output/scenarios_102_30/U-r15-t250000-id4-opi-107_30.csv
Doing model_output/scenarios_102_30/L-r15-t250000-id5-opi-107_30.csv
Doing model_output/scenarios_102_30/U-r150-t250000-id6-opi-107_30.csv
Doing model_output/scenarios_102_30/L-r150-t250000-id7-opi-107_30.csv
Doing model_output/scenarios_102_30/U-r15-t479700-id8-opi-107_30.csv
Doing model_output/scenarios_102_30/L-r15-t479700-id9-opi-107_30.csv
Doing model_output/scenarios_102_30/U-r150-t479700-id10-opi-107_30.csv
Doing model_output/scenarios_102_30/L-r150-t479700-id11-opi-107_30.csv
Doing model_output/scenarios_102_30/U-r15-t1000000-id12-opi-107_30.csv
Doing model_output/scenarios_102_30/L-r15-t1000000-id13-opi-107_30.csv
Doing model_output/sce

In [9]:
scenarios_baseline = {}
for scenario_name, scenario in scenarios.items():
    if scenario_name.split('-')[0] not in scenarios_baseline:
        print(scenario_name)

        maxvaccrate_regional, stockpile_national, stockpile_national_constraint, control_initial = build_scenario(setup, scenario)
        
        p.apply_epicourse(setup, scenario['beta_mult'])


        dosesV = np.zeros_like(dosesV)


        eng.workspace['Vdoses'] = matlab.double(dosesV.tolist())
        eng.workspace['beta_ratio'] = matlab.double(scenario['beta_mult'][0].tolist())
        eng.run('main_script.m', nargout=0)
        ens_exposed = np.array(eng.eval('ens_exposed_preprocess')).flatten()
        
        scenarios_baseline[scenario_name.split('-')[0]] = ens_exposed
        

U-r15-t125000-id0
L-r15-t125000-id1


In [15]:
scn_results

,newdoseperweek,method,infected,post_sample,doses,scenario-beta,scenario-rate,scenario-tot,scenario,bl
0,125000,opi,1.522608e+07,0.0,7.500000e+05,U,r15,t125000,U-r15-t125000-id0,1.525564e+07
1,125000,opi,1.997468e+07,1.0,7.500000e+05,U,r15,t125000,U-r15-t125000-id0,2.000723e+07
2,125000,opi,7.028417e+06,2.0,7.500000e+05,U,r15,t125000,U-r15-t125000-id0,7.035700e+06
3,125000,opi,1.902724e+07,3.0,7.500000e+05,U,r15,t125000,U-r15-t125000-id0,1.907182e+07
4,125000,opi,1.060526e+07,4.0,7.500000e+05,U,r15,t125000,U-r15-t125000-id0,1.061230e+07
...,...,...,...,...,...,...,...,...,...,...
97,1000000,sus,1.690926e+07,97.0,5.272115e+06,U,r150,t1000000,U-r150-t1000000-id14,1.703483e+07
98,1000000,sus,1.557491e+07,98.0,5.272115e+06,U,r150,t1000000,U-r150-t1000000-id14,1.566753e+07
99,1000000,sus,1.641005e+07,99.0,5.272115e+06,U,r150,t1000000,U-r150-t1000000-id14,1.650474e+07
100,1000000,sus,1.420878e+07,100.0,5.272115e+06,U,r150,t1000000,U-r150-t1000000-id14,1.428348e+07


In [17]:
nbtileU = int(len(scn_results.loc[scn_results['scenario-beta'] == 'U', 'scenario-tot'])/len(scenarios_baseline['U']))
nbtileL = int(len(scn_results.loc[scn_results['scenario-beta'] == 'L', 'scenario-tot'])/len(scenarios_baseline['L']))

In [18]:
scn_results['bl'] = 0

scn_results.loc[scn_results['scenario-beta'] == 'U', 'bl'] = np.tile(scenarios_baseline['U'],nbtileU)
scn_results.loc[scn_results['scenario-beta'] == 'L', 'bl'] = np.tile(scenarios_baseline['L'],nbtileL)

In [19]:
scn_results['averted'] = scn_results['bl'] - scn_results['infected']
scn_results['avertedpervacc'] = scn_results['averted']/scn_results['doses']

In [20]:
scn_results.to_csv(f'{generated_dir}/all_summary.csv')